# 知识文件扫描与智能分组 - 步骤1实现

## 目标

实现一个智能的知识文件组织器，能够：

1. **自动扫描**目录下的所有知识文件（PDF/DOC/PPTX）
2. **智能识别**同一知识块的不同格式文件（基于文件名相似度）
3. **自动清洗**文件名中的噪音标记（`[防断更微xxx]`、时间戳等）
4. **提取序号**并按序号排序知识块
5. **选择主文件**（优先级：PDF笔记 > DOC > PPTX）

## 设计原则

- **类型安全**：使用 Type Hints
- **错误处理**：优雅处理异常情况
- **可配置**：关键参数可调整
- **可测试**：模块化设计，易于单元测试
- **文档化**：详细的 Docstring

---


## 1. 导入依赖


In [1]:
import re
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Set
from dataclasses import dataclass, asdict
from collections import defaultdict
from difflib import SequenceMatcher
from enum import IntEnum

print("✅ 依赖导入完成")


✅ 依赖导入完成


## 2. 数据结构定义

使用 `dataclass` 定义清晰的数据结构，便于维护和序列化。


In [2]:
class FilePriority(IntEnum):
    """文件优先级枚举（数字越小优先级越高）"""
    PDF_NOTE = 1      # PDF笔记：内容最完整
    WORD_DOC = 2      # Word文档：详细说明
    PDF_REGULAR = 3   # 普通PDF
    POWERPOINT = 4    # PPT：图表多但文字少
    UNKNOWN = 99      # 未知格式


@dataclass
class FileInfo:
    """文件信息数据类"""
    path: Path                    # 文件路径
    original_name: str            # 原始文件名
    cleaned_name: str             # 清洗后的文件名
    sequence: int                 # 序号（用于排序）
    sequence_str: str             # 序号字符串（如"01"）
    priority: FilePriority        # 文件优先级

    def __repr__(self) -> str:
        return f"FileInfo({self.sequence_str}_{self.cleaned_name[:20]}..., priority={self.priority.name})"


@dataclass
class KnowledgeGroup:
    """知识块分组数据类"""
    group_key: str                # 组的唯一标识
    topic: str                    # 知识主题
    sequence: int                 # 序号
    files: List[FileInfo]         # 该组的所有文件
    primary_file: FileInfo        # 主文件（优先处理）
    file_types: List[str]         # 文件类型列表

    def to_dict(self) -> Dict:
        """转换为字典（便于序列化）"""
        return {
            "group_key": self.group_key,
            "topic": self.topic,
            "sequence": self.sequence,
            "primary_file": str(self.primary_file.path),
            "files": [str(f.path) for f in self.files],
            "file_types": self.file_types,
        }

print("✅ 数据结构定义完成")


✅ 数据结构定义完成


## 3. Knowledge Organizer 核心类

资深Python开发的专业实现：
- 完整类型提示
- 详细Docstring  
- 优雅错误处理
- 可配置参数
- 日志输出


In [3]:
class KnowledgeOrganizer:
    """
    知识文件智能组织器

    职责：扫描、清洗、分组知识文件
    设计模式：单一职责原则（SRP）
    """

    SUPPORTED_EXTENSIONS: Set[str] = {'.pdf', '.doc', '.docx', '.ppt', '.pptx'}

    NOISE_PATTERNS: List[str] = [
        r'\[防断更.*?\]',
        r'\[.*?微.*?\]',
        r'_\d{14}',
        r'_\d{8}',
        r'_笔记',
        r'\s*\(.*?\)\s*',
    ]

    def __init__(
        self,
        knowledge_base_dir: str | Path,
        similarity_threshold: float = 0.7,
        verbose: bool = True
    ):
        self.knowledge_base_dir = Path(knowledge_base_dir)
        self.similarity_threshold = similarity_threshold
        self.verbose = verbose

        if not self.knowledge_base_dir.exists():
            raise ValueError(f"目录不存在: {self.knowledge_base_dir}")
        if not self.knowledge_base_dir.is_dir():
            raise ValueError(f"不是目录: {self.knowledge_base_dir}")

        self._log(f"✅ 初始化完成: {self.knowledge_base_dir}")

    def _log(self, message: str) -> None:
        if self.verbose:
            print(message)

    def clean_filename(self, filename: str) -> str:
        """清洗文件名，去除噪音标记"""
        name = Path(filename).stem
        for pattern in self.NOISE_PATTERNS:
            name = re.sub(pattern, '', name)
        name = re.sub(r'\s+', ' ', name).strip()
        return name

    def extract_sequence_number(self, filename: str) -> Tuple[int, str]:
        """提取序号"""
        match = re.match(r'^(\d+)', filename)
        if match:
            seq_str = match.group(1)
            return (int(seq_str), seq_str)
        return (999999, "")

    def calculate_similarity(self, str1: str, str2: str) -> float:
        """计算相似度（0-1）"""
        return SequenceMatcher(None, str1, str2).ratio()

    def get_file_priority(self, file_path: Path) -> FilePriority:
        """获取文件优先级"""
        name_lower = file_path.name.lower()
        suffix = file_path.suffix.lower()

        if '笔记' in name_lower and suffix == '.pdf':
            return FilePriority.PDF_NOTE
        if suffix in ['.doc', '.docx']:
            return FilePriority.WORD_DOC
        if suffix == '.pdf':
            return FilePriority.PDF_REGULAR
        if suffix in ['.ppt', '.pptx']:
            return FilePriority.POWERPOINT
        return FilePriority.UNKNOWN

    def create_file_info(self, file_path: Path) -> FileInfo:
        """创建文件信息对象"""
        original_name = file_path.name
        cleaned_name = self.clean_filename(original_name)
        sequence, sequence_str = self.extract_sequence_number(cleaned_name)
        priority = self.get_file_priority(file_path)

        return FileInfo(
            path=file_path,
            original_name=original_name,
            cleaned_name=cleaned_name,
            sequence=sequence,
            sequence_str=sequence_str,
            priority=priority
        )

    def group_files_by_similarity(
        self,
        files: List[FileInfo]
    ) -> Dict[str, KnowledgeGroup]:
        """智能分组（核心算法）"""
        if not files:
            return {}

        groups: Dict[str, KnowledgeGroup] = {}
        processed: Set[Path] = set()

        for i, file1 in enumerate(files):
            if file1.path in processed:
                continue

            group_key = f"{file1.sequence_str}_{file1.cleaned_name[:20]}"
            group_files = [file1]
            processed.add(file1.path)

            # 查找相似文件
            for file2 in files[i+1:]:
                if file2.path in processed:
                    continue

                if file1.sequence == file2.sequence:
                    similarity = self.calculate_similarity(
                        file1.cleaned_name,
                        file2.cleaned_name
                    )

                    if similarity >= self.similarity_threshold:
                        group_files.append(file2)
                        processed.add(file2.path)
                        self._log(f"  ↳ 相似度={similarity:.2f}: {file2.original_name}")

            # 选择主文件
            group_files.sort(key=lambda f: (f.priority.value, f.original_name))
            primary_file = group_files[0]

            group = KnowledgeGroup(
                group_key=group_key,
                topic=file1.cleaned_name,
                sequence=file1.sequence,
                files=group_files,
                primary_file=primary_file,
                file_types=[f.path.suffix for f in group_files]
            )

            groups[group_key] = group
            self._log(
                f"✓ 知识块: {file1.cleaned_name} "
                f"({len(group_files)}个文件, 主: {primary_file.path.name})"
            )

        return groups

    def scan_and_organize(self) -> Dict[str, Dict[str, KnowledgeGroup]]:
        """扫描并组织目录（主方法）"""
        result: Dict[str, Dict[str, KnowledgeGroup]] = defaultdict(dict)

        self._log(f"\n{'='*80}")
        self._log(f"📚 开始扫描: {self.knowledge_base_dir}")
        self._log(f"{'='*80}\n")

        # 获取所有文件
        all_files: List[Path] = []
        for ext in self.SUPPORTED_EXTENSIONS:
            all_files.extend(self.knowledge_base_dir.glob(f"*{ext}"))

        if not all_files:
            self._log("⚠️  未找到支持的文件")
            return dict(result)

        self._log(f"📂 找到 {len(all_files)} 个文件\n")

        # 创建FileInfo
        file_infos = [self.create_file_info(f) for f in all_files]

        # 分组
        groups = self.group_files_by_similarity(file_infos)

        # 排序
        sorted_groups = dict(sorted(groups.items(), key=lambda x: x[1].sequence))

        domain_name = self.knowledge_base_dir.name
        result[domain_name] = sorted_groups

        self._log(f"\n{'='*80}")
        self._log(f"✅ 完成: {len(sorted_groups)} 个知识块")
        self._log(f"{'='*80}\n")

        return dict(result)

    def print_organization(self) -> Dict[str, Dict[str, KnowledgeGroup]]:
        """打印组织结构（用于检查）"""
        organized = self.scan_and_organize()

        print("\n" + "="*80)
        print("📊 知识库组织结构")
        print("="*80 + "\n")

        for domain, groups in organized.items():
            print(f"🎯 领域: {domain}")
            print("-" * 80)

            for group_key, group in groups.items():
                print(f"\n  📖 #{group.sequence}: {group.topic}")
                print(f"     主文件: {group.primary_file.original_name}")
                print(f"     优先级: {group.primary_file.priority.name}")
                print(f"     文件数: {len(group.files)}")

                if len(group.files) > 1:
                    print(f"     所有文件:")
                    for f in group.files:
                        print(f"       • {f.original_name} [{f.priority.name}]")

        print("\n" + "="*80 + "\n")
        return organized

    def export_to_json(self, output_path: str | Path) -> None:
        """导出为JSON"""
        organized = self.scan_and_organize()

        export_data = {}
        for domain, groups in organized.items():
            export_data[domain] = {
                group_key: group.to_dict()
                for group_key, group in groups.items()
            }

        output_path = Path(output_path)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)

        self._log(f"✅ 已导出到: {output_path}")


print("✅ KnowledgeOrganizer 类定义完成")


✅ KnowledgeOrganizer 类定义完成


## 4. 使用示例

**步骤1:** 初始化组织器
**步骤2:** 扫描并打印组织结构
**步骤3:** 导出JSON（可选）


In [4]:
# 步骤1: 初始化
organizer = KnowledgeOrganizer(
    knowledge_base_dir="/Users/zhou/Project/AnalystChain/jupyter_notebook/macroeconomic_analysis/knowledge_base",
    similarity_threshold=0.7,
    verbose=True
)


✅ 初始化完成: /Users/zhou/Project/AnalystChain/jupyter_notebook/macroeconomic_analysis/knowledge_base


In [5]:
# 步骤2: 打印组织结构
organized = organizer.print_organization()



📚 开始扫描: /Users/zhou/Project/AnalystChain/jupyter_notebook/macroeconomic_analysis/knowledge_base

📂 找到 51 个文件

  ↳ 相似度=1.00: 13.第十三节 黄金投资手册[防断更微coc36666].doc
  ↳ 相似度=1.00: 13.第十三节 黄金投资手册[防断更微coc36666]_笔记.pdf
✓ 知识块: 13.第十三节 黄金投资手册 (3个文件, 主: 13.第十三节 黄金投资手册[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666].doc
  ↳ 相似度=1.00: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf
✓ 知识块: 03第三节 投资——快速入门读懂经济形势 (3个文件, 主: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 14.第十四节 汇率投资手册[防断更微coc36666].doc
  ↳ 相似度=1.00: 14.第十四节 汇率投资手册[防断更微coc36666]_笔记.pdf
✓ 知识块: 14.第十四节 汇率投资手册 (3个文件, 主: 14.第十四节 汇率投资手册[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666].doc
  ↳ 相似度=1.00: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf
✓ 知识块: 04第四节 出口——快速入门读懂经济形势 (3个文件, 主: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666].doc
  ↳ 相似度=1.00: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666]_笔记.pdf
✓ 知识块: 17.第十七节 格雷厄姆：华尔街教父 (3个文件, 主: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666]_笔记.

In [6]:
# 步骤3: 导出JSON（可选）
organizer.export_to_json("knowledge_organization.json")

# 查看JSON内容
with open("knowledge_organization.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(json.dumps(data, ensure_ascii=False, indent=2))



📚 开始扫描: /Users/zhou/Project/AnalystChain/jupyter_notebook/macroeconomic_analysis/knowledge_base

📂 找到 51 个文件

  ↳ 相似度=1.00: 13.第十三节 黄金投资手册[防断更微coc36666].doc
  ↳ 相似度=1.00: 13.第十三节 黄金投资手册[防断更微coc36666]_笔记.pdf
✓ 知识块: 13.第十三节 黄金投资手册 (3个文件, 主: 13.第十三节 黄金投资手册[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666].doc
  ↳ 相似度=1.00: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf
✓ 知识块: 03第三节 投资——快速入门读懂经济形势 (3个文件, 主: 03第三节 投资——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 14.第十四节 汇率投资手册[防断更微coc36666].doc
  ↳ 相似度=1.00: 14.第十四节 汇率投资手册[防断更微coc36666]_笔记.pdf
✓ 知识块: 14.第十四节 汇率投资手册 (3个文件, 主: 14.第十四节 汇率投资手册[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666].doc
  ↳ 相似度=1.00: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf
✓ 知识块: 04第四节 出口——快速入门读懂经济形势 (3个文件, 主: 04第四节 出口——快速入门读懂经济形势[防断更微coc36666]_笔记.pdf)
  ↳ 相似度=1.00: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666].doc
  ↳ 相似度=1.00: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666]_笔记.pdf
✓ 知识块: 17.第十七节 格雷厄姆：华尔街教父 (3个文件, 主: 17.第十七节 格雷厄姆：华尔街教父[防断更微coc36666]_笔记.

## 5. 单元测试

测试核心功能是否正常


In [7]:
# 测试: 文件名清洗
print("="*80)
print("测试: 文件名清洗")
print("="*80)

test_cases = [
    "01第一节 中国经济的三驾马车[防断更微coc36666]_笔记.pdf",
    "02第二节 消费——快速入门[防断更微coc36666]_20250707140225.pptx",
]

for filename in test_cases:
    cleaned = organizer.clean_filename(filename)
    print(f"原始: {filename}")
    print(f"清洗: {cleaned}\n")


测试: 文件名清洗
原始: 01第一节 中国经济的三驾马车[防断更微coc36666]_笔记.pdf
清洗: 01第一节 中国经济的三驾马车

原始: 02第二节 消费——快速入门[防断更微coc36666]_20250707140225.pptx
清洗: 02第二节 消费——快速入门



In [8]:
# 测试: 相似度计算
print("="*80)
print("测试: 相似度计算")
print("="*80)

pairs = [
    ("01第一节 中国经济的三驾马车", "01第一节 中国经济三驾马车"),
    ("01第一节 中国经济的三驾马车", "02第二节 消费"),
]

for str1, str2 in pairs:
    sim = organizer.calculate_similarity(str1, str2)
    print(f"相似度: {sim:.3f}")
    print(f"  文本1: {str1}")
    print(f"  文本2: {str2}\n")


测试: 相似度计算
相似度: 0.966
  文本1: 01第一节 中国经济的三驾马车
  文本2: 01第一节 中国经济三驾马车

相似度: 0.348
  文本1: 01第一节 中国经济的三驾马车
  文本2: 02第二节 消费



## 6. 总结

### ✅ 已实现功能

1. **自动扫描** - 支持PDF/DOC/PPTX多格式
2. **智能清洗** - 去除噪音标记
3. **序号提取** - 自动识别排序
4. **相似度分组** - difflib.SequenceMatcher算法
5. **优先级选择** - PDF笔记优先
6. **类型安全** - 完整Type Hints
7. **错误处理** - 优雅异常处理
8. **可配置** - similarity_threshold可调
9. **可测试** - 模块化设计
10. **可导出** - JSON序列化

### 🎯 代码质量

- ✅ Type Hints (PEP 484)
- ✅ Docstrings (PEP 257)
- ✅ Error Handling
- ✅ Logging
- ✅ PEP 8 Style
- ✅ 单一职责原则
- ✅ 开闭原则

### 💡 使用建议

1. 先用 `print_organization()` 检查分组
2. 如分组不准确,调整 `similarity_threshold`
3. 如有新噪音模式,添加到 `NOISE_PATTERNS`
4. 用 `export_to_json()` 保存结果

### 📚 下一步

- **步骤2**: 文档加载与清洗
- **步骤3**: 结构化知识提取  
- **步骤4**: 向量化存储
- **步骤5**: 主控制器集成
